In [14]:
from neo4j import GraphDatabase

class Neo4jLoader:
    def __init__(self, uri, user, password):
        self.driver = GraphDatabase.driver(uri, auth=(user, password))

    def close(self):
        self.driver.close()

    def execute_query(self, query, parameters=None):
        with self.driver.session() as session:
            session.run(query, parameters)

    def create_constraints(self):
        constraints = [
            """
            CREATE CONSTRAINT `Civil_Code ID_Civil_Code_ID_uniq` IF NOT EXISTS
            FOR (n: `Civil_Code_ID`) REQUIRE (n.`Civil Code ID`) IS UNIQUE;
            """,
            """
            CREATE CONSTRAINT `Inheritance_Code_Inheritance_Code_uniq` IF NOT EXISTS
            FOR (n: `Inheritance_Code`) REQUIRE (n.`Inheritance Code`) IS UNIQUE;
            """,
            """
            CREATE CONSTRAINT `Court_Finding_Court_Finding_uniq` IF NOT EXISTS
            FOR (n: `Court_Finding`) REQUIRE (n.`Court Finding`) IS UNIQUE;
            """,
            """
            CREATE CONSTRAINT `Court_Opinion_Court_Opinion_uniq` IF NOT EXISTS
            FOR (n: `Court_Opinion`) REQUIRE (n.`Court Opinion`) IS UNIQUE;
            """,
            """
            CREATE CONSTRAINT `Court_Judgment_Court_Judgment_uniq` IF NOT EXISTS
            FOR (n: `Court_Judgment`) REQUIRE (n.`Court Judgment`) IS UNIQUE;
            """,
            """
            CREATE CONSTRAINT `Case_Number_Case_Number_uniq` IF NOT EXISTS
            FOR (n: `Case_Number`) REQUIRE (n.`Case Number`) IS UNIQUE;
            """
        ]
        for constraint in constraints:
            self.execute_query(constraint)
# CALL apoc.load.csv($file_path_root) YIELD map AS row
# LOAD CSV WITH HEADERS FROM $file_path_root AS row # FIELDTERMINATOR ','
    def load_nodes(self, file_path_root):
        queries = [
            # Load Civil_Code_ID nodes
            """
            CALL apoc.import.csv($file_path_root) YIELD map AS row
            WITH row
            WHERE NOT row.`Civil Code ID` IS NULL
            CALL {
                WITH row
                MERGE (n: `Civil_Code_ID` { `Civil Code ID`: toInteger(trim(row.`Civil Code ID`)) })
                SET n.`civil_code_chinese` = row.`Civil Code Chinese Number`
            } IN TRANSACTIONS OF 10000 ROWS;
            """,
            # Load Inheritance_Code nodes
            """
            CALL apoc.import.csv($file_path_root) YIELD map AS row
            WITH row
            WHERE NOT row.`Inheritance Code` IS NULL
            CALL {
                WITH row
                MERGE (n: `Inheritance_Code` { `Inheritance Code`: row.`Inheritance Code` })
            } IN TRANSACTIONS OF 10000 ROWS;
            """
        ]
        for query in queries:
            self.execute_query(query, {"file_path_root": file_path_root})

    def load_relationships(self, file_path_root):
        queries = [
            # HAS_DETAIL relationship
            """
            CALL apoc.import.csv($file_path_root) YIELD map AS row
            WITH row
            MATCH (source: `Civil_Code_ID` { `Civil Code ID`: toInteger(trim(row.`Civil Code ID`)) })
            MATCH (target: `Inheritance_Code` { `Inheritance Code`: row.`Inheritance Code` })
            MERGE (source)-[:HAS_DETAIL]->(target);
            """
        ]
        for query in queries:
            self.execute_query(query, {"file_path_root": file_path_root})


In [15]:
if __name__ == "__main__":
    import pathlib

    # Configure your Neo4j connection details
    URI = "neo4j+s://26f1ead5.databases.neo4j.io"
    USER = "neo4j"
    PASSWORD = "kLVEcuU3_X8fKkTlcYADKGeS2bZsoUtS8oT4Qt8QP7U"
    # FILE_0 = pathlib.Path(r"C:\Users\PC\Documents\NEF6102\Dataset-20241115T102555Z-001\Dataset\Civil Codes.csv").as_uri()
    # FILE_1 = pathlib.Path(r"C:\Users\PC\Documents\NEF6102\Dataset-20241115T102555Z-001\Dataset\normalized_cases.csv").as_uri()
    FILE_0 = "https://github.com/VuTrungNguyen/CMG_law/blob/2595499c1a7cfeb65cfd8ac3b64985645c5484d7/Civil%20Codes.csv"
    FILE_1 = "https://github.com/VuTrungNguyen/CMG_law/blob/2595499c1a7cfeb65cfd8ac3b64985645c5484d7/normalized_cases.csv"

    loader = Neo4jLoader(URI, USER, PASSWORD)

    try:
        # Create constraints
        loader.create_constraints()

        # Load nodes
        loader.load_nodes(FILE_0)

        # Load relationships
        loader.load_relationships(FILE_0)
        loader.load_relationships(FILE_1)

    finally:
        loader.close()

CypherSyntaxError: {code: Neo.ClientError.Statement.SyntaxError} {message: Procedure call does not provide the required number of arguments: got 1 expected at least 3 (total: 3, 0 of which have default values).

Procedure apoc.import.csv has signature: apoc.import.csv(nodes :: LIST<MAP>, rels :: LIST<MAP>, config :: MAP) :: file :: STRING, source :: STRING, format :: STRING, nodes :: INTEGER, relationships :: INTEGER, properties :: INTEGER, time :: INTEGER, rows :: INTEGER, batchSize :: INTEGER, batches :: INTEGER, done :: BOOLEAN, data :: ANY
meaning that it expects at least 3 arguments of types LIST<MAP>, LIST<MAP>, MAP
Description: Imports `NODE` and `RELATIONSHIP` values with the given labels and types from the provided CSV file. (line 2, column 13 (offset: 13))
"            CALL apoc.import.csv($file_path_root) YIELD map AS row"
             ^}

In [38]:
import csv
from neo4j import GraphDatabase

# Neo4j connection settings
URI = "neo4j+s://26f1ead5.databases.neo4j.io"  # Replace with your Neo4j URI
USERNAME = "neo4j"  # Replace with your Neo4j username
PASSWORD = "kLVEcuU3_X8fKkTlcYADKGeS2bZsoUtS8oT4Qt8QP7U"  # Replace with your Neo4j password

# Initialize Neo4j driver
driver = GraphDatabase.driver(URI, auth=(USERNAME, PASSWORD))

# Function to create constraints and full-text index in Neo4j
def create_constraints(tx):
    constraints = [
        "CREATE CONSTRAINT IF NOT EXISTS FOR (n: Civil_Code_ID) REQUIRE n.`Civil Code ID` IS UNIQUE",
        "CREATE CONSTRAINT IF NOT EXISTS FOR (n: Inheritance_Code) REQUIRE n.`Inheritance Code` IS UNIQUE",
        "CREATE CONSTRAINT IF NOT EXISTS FOR (n: Case_Number) REQUIRE n.`Case Number` IS UNIQUE"
    ]
    
    # Create full-text index for large text fields like 'Court Finding'
    create_fulltext_index_query = """
    CREATE FULLTEXT INDEX Court_Finding_Fulltext FOR (n:Court_Finding) ON EACH [n.`Court Finding`]
    """
    constraints.append(create_fulltext_index_query)

    for constraint in constraints:
        tx.run(constraint)

# Function to load nodes and relationships into Neo4j from CSV files
def load_nodes_and_relationships(tx, file_path_root, file_0, file_1):
    initial_query = "CALL apoc.schema.assert({}, {})"
    tx.run(initial_query)
    # Function to load CSV data and create nodes in the database
    def load_csv_and_create_nodes(tx, file_path, node_label, id_column, fields):
        with open(file_path, "r", encoding="utf-8") as file:
            reader = csv.DictReader(file)
            for row in reader:
                # Extract ID value and properties
                id_value = row[id_column]
                node_props = {field: row[field] for field in fields if field in row}
                
                # Handle large text property, e.g., Court Finding
                if "Court Finding" in node_props:
                    node_props["Court Finding"] = node_props["Court Finding"][:4096]  # Truncate if needed

                # Create the node
                cypher_query = f"""
                MERGE (n:{node_label} {{`{id_column}`: $id_value}})
                SET n += $node_props
                """
                tx.run(cypher_query, id_value=id_value, node_props=node_props)

                # Create separate node for large data like Court Finding (optional)
                if "Court Finding" in node_props:
                    large_data_node_query = """
                    CREATE (m:LargeData {data: $large_data})
                    MERGE (n)-[:HAS_LARGE_DATA]->(m)
                    """
                    tx.run(large_data_node_query, large_data=row["Court Finding"])
                    
    # Step 3: Execute in your session
    load_csv_and_create_nodes(tx, f"{file_0}", "Civil_Code_ID", "Civil Code ID", 
                              ["Civil Code ID", "Civil Code Chinese Number", "Title"])
    load_csv_and_create_nodes(tx, f"{file_0}", "Inheritance_Code", "Inheritance Code", ["Inheritance Code"])
    
    # Load Court-related nodes
    load_csv_and_create_nodes(tx, f"{file_1}", "Court_Finding", "Court Finding", ["Court Finding"])
    load_csv_and_create_nodes(tx, f"{file_1}", "Court_Opinion", "Court Opinion", ["Court Opinion"])
    load_csv_and_create_nodes(tx, f"{file_1}", "Court_Judgment", "Court Judgment", ["Court Judgment"])
    load_csv_and_create_nodes(tx, f"{file_1}", "Case_Number", "Case Number", ["Case Number", "Raw Text", "Link"])

    # Function to load relationships between nodes from CSV data
    def load_relationships(file_name, source_label, source_id_column, target_label, target_id_column, relationship_type):
        with open(f"{file_name}", newline='', encoding='utf-8') as csvfile:
            reader = csv.DictReader(csvfile)
            for row in reader:
                source_id = row[source_id_column].strip()
                target_id = row[target_id_column].strip()
                if source_id and target_id:
                    cypher_query = f"""
                    MATCH (source:{source_label} {{{source_id_column}: $source_id}})
                    MATCH (target:{target_label} {{{target_id_column}: $target_id}})
                    MERGE (source)-[r:{relationship_type}]->(target)
                    """
                    tx.run(cypher_query, source_id=source_id, target_id=target_id)

    # Load relationships between different entities
    load_relationships(file_0, "Civil_Code_ID", "Civil Code ID", "Inheritance_Code", "Inheritance Code", "HAS_DETAIL")
    load_relationships(file_1, "Case_Number", "Case Number", "Court_Finding", "Court Finding", "HAS_COURT_FINDING")
    load_relationships(file_1, "Case_Number", "Case Number", "Civil_Code_ID", "Civil Code ID", "HAS_CIVIL_CODE")
    load_relationships(file_1, "Case_Number", "Case Number", "Court_Opinion", "Court Opinion", "HAS_COURT_OPINION")
    load_relationships(file_1, "Case_Number", "Case Number", "Court_Judgment", "Court Judgment", "HAS_COURT_JUDGEMENT")

# Define file paths
file_path_root = ""  # Update with the actual path on your system
file_0 = r"C:\Users\PC\Documents\NEF6102\Dataset-20241115T102555Z-001\Dataset\Civil Codes.csv"
file_1 = r"C:\Users\PC\Documents\NEF6102\Dataset-20241115T102555Z-001\Dataset\normalized_cases.csv"

# Start a session and run the script
with driver.session() as session:
    session.write_transaction(create_constraints)
    session.write_transaction(load_nodes_and_relationships, file_path_root, file_0, file_1)

# Close the driver after use
driver.close()


C:\Users\PC\AppData\Local\Temp\ipykernel_12568\585607655.py:101: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(create_constraints)
C:\Users\PC\AppData\Local\Temp\ipykernel_12568\585607655.py:102: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(load_nodes_and_relationships, file_path_root, file_0, file_1)


Forbidden: {code: Neo.ClientError.Transaction.ForbiddenDueToTransactionType} {message: Cannot perform data updates in a transaction that has performed schema updates.}

In [43]:
import csv
from neo4j import GraphDatabase

# Neo4j connection settings
URI = "neo4j+s://26f1ead5.databases.neo4j.io"  # Replace with your Neo4j URI
USERNAME = "neo4j"  # Replace with your Neo4j username
PASSWORD = "kLVEcuU3_X8fKkTlcYADKGeS2bZsoUtS8oT4Qt8QP7U"  # Replace with your Neo4j password

# Initialize Neo4j driver
driver = GraphDatabase.driver(URI, auth=(USERNAME, PASSWORD))

# Function to create constraints and full-text index in Neo4j
def create_constraints(tx):
    constraints = [
        "CREATE CONSTRAINT IF NOT EXISTS FOR (n: Civil_Code_ID) REQUIRE n.`Civil Code ID` IS UNIQUE",
        "CREATE CONSTRAINT IF NOT EXISTS FOR (n: Inheritance_Code) REQUIRE n.`Inheritance Code` IS UNIQUE",
        "CREATE CONSTRAINT IF NOT EXISTS FOR (n: Case_Number) REQUIRE n.`Case Number` IS UNIQUE"
    ]
    
    # Create full-text index for large text fields like 'Court Finding'
    create_fulltext_index_query = """
    CREATE FULLTEXT INDEX Court_Finding_Fulltext FOR (n:Court_Finding) ON EACH [n.`Court Finding`]
    """
    constraints.append(create_fulltext_index_query)

    for constraint in constraints:
        tx.run(constraint)

# Function to load nodes and relationships into Neo4j from CSV files
def load_nodes_and_relationships(tx, file_path_root, file_0, file_1):

    # Function to load CSV data and create nodes in the database
    def load_csv_and_create_nodes(tx, file_path, node_label, id_column, fields):
        with open(file_path, "r", encoding="utf-8") as file:
            reader = csv.DictReader(file)
            for row in reader:
                # Extract ID value and properties
                id_value = row[id_column]
                node_props = {field: row[field] for field in fields if field in row}
                
                # Handle large text property, e.g., Court Finding
                if "Court Finding" in node_props:
                    node_props["Court Finding"] = node_props["Court Finding"][:4096]  # Truncate if needed

                # Create the node
                cypher_query = f"""
                MERGE (n:{node_label} {{`{id_column}`: $id_value}})
                SET n += $node_props
                """
                tx.run(cypher_query, id_value=id_value, node_props=node_props)

                # Create separate node for large data like Court Finding (optional)
                if "Court Finding" in node_props:
                    large_data_node_query = """
                    CREATE (m:LargeData {data: $large_data})
                    MERGE (n)-[:HAS_LARGE_DATA]->(m)
                    """
                    tx.run(large_data_node_query, large_data=row["Court Finding"])
                    
    # Step 3: Execute in your session
    load_csv_and_create_nodes(tx, f"{file_0}", "Civil_Code_ID", "Civil Code ID", 
                              ["Civil Code ID", "Civil Code Chinese Number", "Title"])
    load_csv_and_create_nodes(tx, f"{file_0}", "Inheritance_Code", "Inheritance Code", ["Inheritance Code"])
    
    # Load Court-related nodes
    load_csv_and_create_nodes(tx, f"{file_1}", "Court_Finding", "Court Finding", ["Court Finding"])
    load_csv_and_create_nodes(tx, f"{file_1}", "Court_Opinion", "Court Opinion", ["Court Opinion"])
    load_csv_and_create_nodes(tx, f"{file_1}", "Court_Judgment", "Court Judgment", ["Court Judgment"])
    load_csv_and_create_nodes(tx, f"{file_1}", "Case_Number", "Case Number", ["Case Number", "Raw Text", "Link"])

    # Function to load relationships between nodes from CSV data
    def load_relationships(file_name, source_label, source_id_column, target_label, target_id_column, relationship_type):
        with open(f"{file_name}", newline='', encoding='utf-8') as csvfile:
            reader = csv.DictReader(csvfile)
            for row in reader:
                source_id = row[source_id_column].strip()
                target_id = row[target_id_column].strip()
                if source_id and target_id:
                    cypher_query = f"""
                    MATCH (source:{source_label} {{`{source_id_column}`: $source_id}})
                    MATCH (target:{target_label} {{`{target_id_column}`: $target_id}})
                    MERGE (source)-[r:{relationship_type}]->(target)
                    """
                    tx.run(cypher_query, source_id=source_id, target_id=target_id)

    # Load relationships between different entities
    load_relationships(file_0, "Civil_Code_ID", "Civil Code ID", "Inheritance_Code", "Inheritance Code", "HAS_DETAIL")
    load_relationships(file_1, "Case_Number", "Case Number", "Court_Finding", "Court Finding", "HAS_COURT_FINDING")
    load_relationships(file_1, "Case_Number", "Case Number", "Civil_Code_ID", "Civil Code ID", "HAS_CIVIL_CODE")
    load_relationships(file_1, "Case_Number", "Case Number", "Court_Opinion", "Court Opinion", "HAS_COURT_OPINION")
    load_relationships(file_1, "Case_Number", "Case Number", "Court_Judgment", "Court Judgment", "HAS_COURT_JUDGEMENT")

# Define file paths
file_path_root = ""  # Update with the actual path on your system
file_0 = r"C:\Users\PC\Documents\NEF6102\Dataset-20241115T102555Z-001\Dataset\Civil Codes.csv"
file_1 = r"C:\Users\PC\Documents\NEF6102\Dataset-20241115T102555Z-001\Dataset\normalized_cases.csv"

# Start a session and run the script
with driver.session() as session:
    # First, create the constraints and full-text index
    session.write_transaction(create_constraints)
    
    # Then, load the nodes and relationships
    session.write_transaction(load_nodes_and_relationships, file_path_root, file_0, file_1)

# Close the driver after use
driver.close()


C:\Users\PC\AppData\Local\Temp\ipykernel_12568\2743516795.py:101: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(create_constraints)
C:\Users\PC\AppData\Local\Temp\ipykernel_12568\2743516795.py:104: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(load_nodes_and_relationships, file_path_root, file_0, file_1)


# Imports and Setup

In [ ]:
import csv
from neo4j import GraphDatabase

# Neo4j connection settings
URI = "neo4j+s://26f1ead5.databases.neo4j.io"  # Replace with your Neo4j URI
USERNAME = "neo4j"  # Replace with your Neo4j username
PASSWORD = "kLVEcuU3_X8fKkTlcYADKGeS2bZsoUtS8oT4Qt8QP7U"  # Replace with your Neo4j password

# Initialize Neo4j driver
driver = GraphDatabase.driver(URI, auth=(USERNAME, PASSWORD))


# Create Constraints Function

In [ ]:
def create_constraints(tx):
    constraints = [
        "CREATE CONSTRAINT IF NOT EXISTS FOR (n: Civil_Code_ID) REQUIRE n.`Civil Code ID` IS UNIQUE",
        "CREATE CONSTRAINT IF NOT EXISTS FOR (n: Inheritance_Code) REQUIRE n.`Inheritance Code` IS UNIQUE",
        "CREATE CONSTRAINT IF NOT EXISTS FOR (n: Case_Number) REQUIRE n.`Case Number` IS UNIQUE"
    ]
    
    # Create full-text index for large text fields like 'Court Finding'
    create_fulltext_index_query = """
    CREATE FULLTEXT INDEX Court_Finding_Fulltext FOR (n:Court_Finding) ON EACH [n.`Court Finding`]
    """
    constraints.append(create_fulltext_index_query)

    for constraint in constraints:
        print(f"Running constraint: {constraint}")  # Debugging line
        tx.run(constraint)


# Load Nodes Function

In [ ]:
def load_nodes_and_relationships(tx, file_path_root, file_0, file_1):
    # Function to load CSV data and create nodes in the database
    def load_csv_and_create_nodes(tx, file_path, node_label, id_column, fields):
        with open(file_path, "r", encoding="utf-8") as file:
            reader = csv.DictReader(file)
            for row in reader:
                # Extract ID value and properties
                id_value = row[id_column]
                node_props = {field: row[field] for field in fields if field in row}
                
                # Handle large text property, e.g., Court Finding
                if "Court Finding" in node_props:
                    node_props["Court Finding"] = node_props["Court Finding"][:4096]  # Truncate if needed

                # Create the node
                cypher_query = f"""
                MERGE (n:{node_label} {{`{id_column}`: $id_value}})
                SET n += $node_props
                """
                print(f"Running query: {cypher_query}")  # Debugging line
                tx.run(cypher_query, id_value=id_value, node_props=node_props)

                # Create separate node for large data like Court Finding (optional)
                if "Court Finding" in node_props:
                    large_data_node_query = """
                    CREATE (m:LargeData {data: $large_data})
                    MERGE (n)-[:HAS_LARGE_DATA]->(m)
                    """
                    print(f"Running query for large data: {large_data_node_query}")  # Debugging line
                    tx.run(large_data_node_query, large_data=row["Court Finding"])
    
    # Load nodes from the CSV files
    load_csv_and_create_nodes(tx, f"{file_0}", "Civil_Code_ID", "Civil Code ID", 
                              ["Civil Code ID", "Civil Code Chinese Number", "Title"])
    load_csv_and_create_nodes(tx, f"{file_0}", "Inheritance_Code", "Inheritance Code", ["Inheritance Code"])
    load_csv_and_create_nodes(tx, f"{file_1}", "Court_Finding", "Court Finding", ["Court Finding"])
    load_csv_and_create_nodes(tx, f"{file_1}", "Court_Opinion", "Court Opinion", ["Court Opinion"])
    load_csv_and_create_nodes(tx, f"{file_1}", "Court_Judgment", "Court Judgment", ["Court Judgment"])
    load_csv_and_create_nodes(tx, f"{file_1}", "Case_Number", "Case Number", ["Case Number", "Raw Text", "Link"])


# Load Relationships Function

In [ ]:
def load_relationships(file_name, source_label, source_id_column, target_label, target_id_column, relationship_type):
    with open(f"{file_name}", newline='', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            source_id = row[source_id_column].strip()
            target_id = row[target_id_column].strip()
            if source_id and target_id:
                cypher_query = f"""
                MATCH (source:{source_label} {{{source_id_column}: $source_id}})
                MATCH (target:{target_label} {{{target_id_column}: $target_id}})
                MERGE (source)-[r:{relationship_type}]->(target)
                """
                print(f"Running relationship query: {cypher_query}")  # Debugging line
                tx.run(cypher_query, source_id=source_id, target_id=target_id)


# Main Execution

In [ ]:
# Define file paths
file_path_root = ""  # Update with the actual path on your system
file_0 = r"C:\Users\PC\Documents\NEF6102\Dataset-20241115T102555Z-001\Dataset\Civil Codes.csv"
file_1 = r"C:\Users\PC\Documents\NEF6102\Dataset-20241115T102555Z-001\Dataset\normalized_cases.csv"

# Start a session and run the script
with driver.session() as session:
    # First, create the constraints and full-text index
    session.write_transaction(create_constraints)
    
    # Then, load the nodes and relationships
    session.write_transaction(load_nodes_and_relationships, file_path_root, file_0, file_1)
    session.write_transaction(load_relationships, file_0, "Civil_Code_ID", "Civil Code ID", "Inheritance_Code", "Inheritance Code", "HAS_DETAIL")
    session.write_transaction(load_relationships, file_1, "Case_Number", "Case Number", "Court_Finding", "Court Finding", "HAS_COURT_FINDING")
    session.write_transaction(load_relationships, file_1, "Case_Number", "Case Number", "Civil_Code_ID", "Civil Code ID", "HAS_CIVIL_CODE")
    session.write_transaction(load_relationships, file_1, "Case_Number", "Case Number", "Court_Opinion", "Court Opinion", "HAS_COURT_OPINION")
    session.write_transaction(load_relationships, file_1, "Case_Number", "Case Number", "Court_Judgment", "Court Judgment", "HAS_COURT_JUDGEMENT")
